In [11]:
!pip install faiss-cpu==1.7.4
!pip install sentence-transformers

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


# Step 1: Create source embeddings for the text column

In [9]:
from sentence_transformers import SentenceTransformer

C:\Users\ACER\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [13]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")
vectors = encoder.encode(df.text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\ACER\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
vectors.shape

(8, 384)

In [16]:
dim = vectors.shape[1]
dim

384

# Step 2: Build a FAISS Index for vectors

In [18]:
import faiss
index = faiss.IndexFlatL2(dim)

In [19]:
index.add(vectors)

In [20]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000026B4BB17660> >

# Step 3: Normalize the source vectors(as we are using L2 distance to measure similarity) and add to the index

In [21]:
index.add(vectors)

In [22]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000026B4BB17660> >

# Step 4: Encode search text using same encoder and normalize the output vector

In [24]:
search_query = "Looking for places to visit during holidays"
vec = encoder.encode(search_query)
vec.shape

(384,)

In [25]:
import numpy as np
svec = np.array(vec).reshape(1, -1)
svec.shape

(1, 384)

# Step 5: Search for similar vectors in the FAISS index created

In [42]:
distances, I = index.search(svec, k=1)
distances

array([[0.9482849]], dtype=float32)

In [43]:
I

array([[6]], dtype=int64)

In [44]:
I.tolist()

[[6]]

In [45]:
row_indices = I.tolist()[0]
row_indices

[6]

In [46]:
df.loc[row_indices]

,text,category
6,Exciting vacation destinations for your next trip,Travel
